In [1]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata, rrankdata
from pymcdm import weights as mcdm_weights

In [2]:
from prettytable import from_csv
with open("breast_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+---------+-----------+--------+-------------+----------+---------+---------+-------------+--------------+--------------+
|                      | ACURACY | PRECISION | RECALL |     AUC     | F1-SCORE | FP_RATE | FN_RATE | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+---------+-----------+--------+-------------+----------+---------+---------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.95  |    0.95   |  0.95  | 0.991216216 |   0.95   |  0.075  |  0.054  |    0.925    | 0.052631579  | 4.763969158  |
|    DECISION TREE     |   0.95  |    0.95   |  0.95  | 0.947972973 |   0.95   |   0.05  |  0.054  |     0.95    | 0.052631579  | 4.763969158  |
| GAUSSIAN NAIVE BAYES |   0.94  |    0.94   |  0.94  | 0.988851351 |   0.94   |  0.125  |  0.027  |    0.875    | 0.061403509  | 4.763969158  |
|         SVM          |   0.44  |    0.42   |  0.44  | 0.755743243 |   0.43   |   0.85  |  0.4054 |     0.15    | 0.561403509  | 

In [3]:
x_matrix, alt_names = mcdm.load("breast_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weights = mcdm_weights.equal(alts)
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [4]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


3.0 0.7375491911067369
4.0 0.7319194107786223
5.0 0.7305643278497282
7.0 0.2307148442838144
6.0 0.3852283947858865
2.0 0.7500000000000001
1.0 0.8580470522468049


In [5]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2=rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

3.0 0.5312643849820373
4.0 0.5394425381555598
5.0 0.542863104093194
7.0 1.0
6.0 0.91975577601531
2.0 0.5
1.0 0.009804763950867086


In [6]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

2.0 0.31666666666666665
3.0 0.28333333333333327
4.0 -0.08333333333333331
7.0 -0.7166666666666666
6.0 -0.44999999999999996
1.0 0.8333333333333333
5.0 -0.1833333333333334


In [7]:
from prettytable import PrettyTable
print("#########################BREAST DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################BREAST DATASET#######################
+----------------------+--------+--------------------+-------+----------------------+-----------+----------------------+
|        Model         | TOPSIS |       Value        | VIKOR |        VALUES        | PROMETHEE |        VALUES        |
+----------------------+--------+--------------------+-------+----------------------+-----------+----------------------+
| LOGISTIC REGRESSION  |  3.0   | 0.7375491911067369 |  3.0  |  0.5312643849820373  |    2.0    | 0.31666666666666665  |
|    DECISION TREE     |  4.0   | 0.7319194107786223 |  4.0  |  0.5394425381555598  |    3.0    | 0.28333333333333327  |
| GAUSSIAN NAIVE BAYES |  5.0   | 0.7305643278497282 |  5.0  |  0.542863104093194   |    4.0    | -0.08333333333333331 |
|         SVM          |  7.0   | 0.2307148442838144 |  7.0  |         1.0          |    7.0    | -0.7166666666666666  |
|    NEURAL NETWORK    |  6.0   | 0.3852283947858865 |  6.0  |   0.91975577601531   |    6

In [8]:
ans=mcdm.rank(x_matrix, alt_names=alt_names, n_method="Linear2", w_method="CRITIC", s_method="SAW") 
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
|    RANDOM FOREST     | 0.6107459811888937 |
| LOGISTIC REGRESSION  | 0.6087143889491596 |
| GAUSSIAN NAIVE BAYES | 0.6039820058404186 |
|    DECISION TREE     | 0.5941742676105118 |
|    NEURAL NETWORK    | 0.5163670401408708 |
|     LINEAR MODEL     | 0.5133279573527112 |
|         SVM          | 0.4782700946309548 |
+----------------------+--------------------+


In [9]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,n_method="Linear1",w_method="CRITIC",s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
|         SVM          |  0.545420093059679  |
|     LINEAR MODEL     |  0.3663260784445677 |
| LOGISTIC REGRESSION  | 0.27672590978974165 |
| GAUSSIAN NAIVE BAYES |  0.274542712549622  |
|    DECISION TREE     | 0.26070320399710223 |
|    NEURAL NETWORK    |         0.0         |
|    RANDOM FOREST     |         0.0         |
+----------------------+---------------------+
